# Abalone Age Prediction by Multiclass Classification using ANN (testing)

## Load dataset for the training process

In [1]:
import numpy as np
import pandas as pd

### Open the CSV

In [2]:
df_test = pd.read_csv("abalone_test.csv")
print("Num test:", len(df_test))

Num test: 835


### Create dataset loader for testing process

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset

### Features to be used and target

In [4]:
x_names = ["Sex", "Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight"]
y_name = "Rings"
# y_classes = list(range(1, 30)) # 1 to 29 (from EDA)

### Convert Pandas dataframe to PyTorch dataset

In [5]:
def df_to_dataset(df: pd.DataFrame) -> Dataset:
    features = df[x_names].to_numpy(dtype=np.float32)
    # labels = df[[y_name]].to_numpy(dtype=np.int64).squeeze(axis=1)
    # labels = labels - 1 # map class #0 to Rings 1
    labels = df[[y_name]].to_numpy(dtype=np.float32).squeeze(axis=1)
    features = torch.from_numpy(features)
    labels = torch.from_numpy(labels)
    my_dataset = TensorDataset(features, labels)
    return my_dataset

In [6]:
ds_test = df_to_dataset(df_test)

### Create PyTorch data loader

In [7]:
BATCH_SIZE = 64
loader_test = DataLoader(ds_test, batch_size=BATCH_SIZE)

## Model testing

In [8]:
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm
# from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

### Which device we will use for testing process (CPU/GPU)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


### Create the model

In [10]:
class MLP4Layers(nn.Module):
    def __init__(self, n_features: int, n_classes: int):
        super().__init__()
        self.fc1 = nn.Linear(n_features, 128)
        self.fc2 = nn.Linear(128, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# model = MLP4Layers(n_features=len(x_names), n_classes=len(y_classes))
model = MLP4Layers(n_features=len(x_names), n_classes=1)
# Move the model from CPU to the device
# Actually, only required if the device is not CPU and has no effect if it is CPU
model = model.to(device)

### Load the trained weights of the model parameters

In [11]:
WEIGHT_PATH = "runs_reg/train_20221013_190916/best.pt"
model.load_state_dict(torch.load(WEIGHT_PATH))

<All keys matched successfully>

### The testing process

In [12]:
# We don't need gradients for the model testing process
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for i, vdata in tqdm(enumerate(loader_test)):
        vinputs, vlabels = vdata
        y_true.extend(vlabels.numpy().tolist())
        voutputs = model(vinputs.to(device))
        # the class with the highest energy is what we choose as prediction
        # _, predicted = torch.max(voutputs.data, 1)
        # y_pred.extend(predicted.cpu().numpy().tolist())
        # we save the regression result as is
        y_pred.extend(voutputs.cpu().numpy().tolist())

# Calculate our classification metrics
# acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred, average="weighted")
# print(f'Accuracy: {acc}, Weighted F1: {f1}')

# Calculate our regression metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)
print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R2: {r2}")

0it [00:00, ?it/s]

MAE: 1.500287316088191, MSE: 4.494114216488882, RMSE: 2.119932597157014, R2: 0.5182925360623976
